# Implementacja eksperckiego systemu dla zwierząt

Przykład z [Kursu AI dla początkujących](http://github.com/microsoft/ai-for-beginners).

W tym przykładzie zaimplementujemy prosty system oparty na wiedzy służący do rozpoznawania zwierząt na podstawie niektórych cech fizycznych. System można przedstawić za pomocą następującego drzewa AND-OR (to jest część całego drzewa, możemy łatwo dodać więcej reguł):

![](../../../../../../translated_images/pl/AND-OR-Tree.5592d2c70187f283.webp)


## Nasza własna powłoka eksperckich systemów z wnioskowaniem wstecznym

Spróbujmy zdefiniować prosty język do reprezentacji wiedzy oparty na regułach produkcyjnych. Użyjemy klas Pythona jako słów kluczowych do definiowania reguł. W zasadzie będą trzy typy klas:
* `Ask` reprezentuje pytanie, które należy zadać użytkownikowi. Zawiera zbiór możliwych odpowiedzi.
* `If` reprezentuje regułę, i jest to tylko składniowy cukier do przechowywania treści reguły
* `AND`/`OR` to klasy reprezentujące gałęzie drzewa typu AND/OR. Przechowują one listę argumentów wewnątrz. Aby uprościć kod, cała funkcjonalność jest zdefiniowana w klasie bazowej `Content`


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

W naszym systemie pamięć robocza zawierałaby listę **faktów** jako **pary atrybut-wartość**. Bazę wiedzy można zdefiniować jako jeden duży słownik, który mapuje akcje (nowe fakty, które powinny zostać wstawione do pamięci roboczej) na warunki, wyrażone jako wyrażenia AND-OR. Ponadto niektóre fakty mogą być `Ask`-owane.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Aby wykonać wnioskowanie wsteczne, zdefiniujemy klasę `Knowledgebase`. Będzie ona zawierać:
* Roboczą `memory` - słownik mapujący atrybuty na wartości
* Zasady bazy wiedzy `rules` w formacie zdefiniowanym powyżej

Dwie główne metody to:
* `get` do uzyskania wartości atrybutu, wykonująca wnioskowanie w razie potrzeby. Na przykład, `get('color')` pobierze wartość pola koloru (zapytuje, jeśli trzeba, i zapisuje wartość do późniejszego użycia w pamięci roboczej). Jeśli zapytamy o `get('color:blue')`, zapyta o kolor, a następnie zwróci wartość `y`/`n` w zależności od koloru.
* `eval` wykonuje faktyczne wnioskowanie, tzn. przeszukuje drzewo AND/OR, ocenia podcele itd.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Teraz zdefiniujmy naszą bazę wiedzy o zwierzętach i przeprowadźmy konsultację. Zauważ, że to wywołanie będzie zadawać pytania. Możesz odpowiadać, wpisując `y`/`n` dla pytań tak/nie lub podając numer (0..N) dla pytań z dłuższymi odpowiedziami do wyboru.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Korzystanie z Experta do wnioskowania progresywnego

W następnym przykładzie spróbujemy zaimplementować wnioskowanie progresywne, korzystając z jednej z bibliotek do reprezentacji wiedzy, [Experta](https://github.com/nilp0inter/experta). **Experta** to biblioteka do tworzenia systemów wnioskowania progresywnego w Pythonie, zaprojektowana tak, aby była podobna do klasycznego, starego systemu [CLIPS](http://www.clipsrules.net/index.html).

Moglibyśmy także sami zaimplementować wnioskowanie progresywne bez większych problemów, ale naiwne implementacje zazwyczaj nie są zbyt efektywne. Do bardziej efektywnego dopasowywania reguł używany jest specjalny algorytm [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Zdefiniujemy nasz system jako klasę dziedziczącą po `KnowledgeEngine`. Każda reguła jest definiowana przez osobną funkcję z adnotacją `@Rule`, która określa, kiedy reguła powinna zostać wywołana. Wewnątrz reguły możemy dodać nowe fakty za pomocą funkcji `declare`, a dodanie tych faktów spowoduje wywołanie kolejnych reguł przez silnik wnioskowania w przód.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Gdy zdefiniujemy bazę wiedzy, wprowadzamy do naszej pamięci roboczej kilka początkowych faktów, a następnie wywołujemy metodę `run()`, aby przeprowadzić wnioskowanie. W efekcie można zauważyć, że nowe wnioskowane fakty są dodawane do pamięci roboczej, w tym ostateczny fakt dotyczący zwierzęcia (jeśli poprawnie ustawiliśmy wszystkie początkowe fakty).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mimo że staramy się zapewnić dokładność, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub niedokładności. Oryginalny dokument w języku źródłowym powinien być uznawany za autorytatywne źródło. W przypadku informacji o krytycznym znaczeniu zalecane jest skorzystanie z profesjonalnego tłumaczenia wykonanego przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
